In [ ]:
from prefect import Flow, tags, task
from time import sleep
import random
import sys

# --------
# assign tags at task creation
# --------

@task(tags=["print"])
def say_hi(name):
    """Prints `name` 10 times."""
    pause_at = random.randint(0, 10)
    for i in range(10):
        if i == pause_at:
            sleep(1)
        print(f'{name} was here')
        sys.stdout.flush()
        
# --------
# or at flow creation
# --------
        
with Flow() as f:
    with tags("example"):
        a = say_hi("alice")
        b = say_hi("bob")
    c = say_hi("chris")

a.tags # {'example', 'print'}
c.tags # {'print'}


In [ ]:
with Flow(throttle={'print': 1}) as f:
    ...


In [ ]:
f.run(throttle={'print': 1})


In [ ]:
f.run()
# Success("All reference tasks succeeded.")

# chris was here
# ... 9 more times ...
# bob was here
# ... 9 more times ...
# alice was here
# ... 9 more times ...


In [ ]:
from prefect.engine.executors import DaskExecutor

f.run(executor=DaskExecutor(processes=True))
# Success("All reference tasks succeeded.")


In [ ]:
f.run(executor=DaskExecutor(processes=True), throttle={'print': 1})
# Success("All reference tasks succeeded.")


In [ ]:
f.run(executor=DaskExecutor(processes=True), throttle={'print': 2})
# Success("All reference tasks succeeded.")


In [ ]:
f.run(executor=DaskExecutor(processes=True), throttle={'example': 1})
# Success("All reference tasks succeeded.")


In [ ]:
with Flow() as big_flow:
    with tags("group C"):
        a = say_hi("alice")
        b = say_hi("bob")
    with tags("group B"):
        c = say_hi("chris")
        with tags("group A"):
            d = say_hi("jeremiah")
            e = say_hi("josh")
            
e.tags # {"group A", "group B", "group C"}

# --------
# run the flow
# --------

big_flow.run(executor=DaskExecutor(processes=True),
             throttle={'group A': 1, 'group B': 2, 'group C': 1})
# Success("All reference tasks succeeded.")
